## Question Generator Using API

1. This is code I generated
2. It generates a minimum amount of questions that are relevant
3. It queries LLM Studio or any LLM that has an OpenAI compatible API
4. I'm using OpenAI library but not OpenAI Service
5. It fails to generate questions after some time
6. It does not understand the instructions sometimes
7. Could have potential
8. Super slow, ran for 2 hours and it generated just a few questions and answers

In [1]:
%reload_ext autoreload
%autoreload 2

import sys

sys.path.append(".")
sys.path.append("..")
sys.path.append("../..")

In [2]:
!{sys.executable} -m pip install openai

In [3]:
import os

dir_cur = os.getcwd()
pdf_file = "fibra-summary.mmd"
input_file = os.path.join(f"{dir_cur}/output/", pdf_file)

In [4]:
content_lines = open(input_file, "r").readlines()

In [28]:
import csv

def write_csv(file_path, data, write_headers=False):
  print("Data", data)
  file_path = os.path.join(f"{dir_cur}/output/", file_name)
  with open(file_path, mode='w', newline='') as file:
    writer = csv.writer(file)

    if write_headers:
      writer.writerow(["Context", "Question", "Answer", "Certainty"])
    
    if len(data) == 0:
      return
    
    column = data[0].split("\n")
    print(column)
    writer.writerow({
      "Context": column[0],
      "Question": column[1],
      "Answer": column[2],
      "Certainty": column[3]
    }
    )

def write_to_file(file_path, data):
  with open(file_path, mode='w') as file:
    file.write(data)

def append_to_file(file_path, data):
  with open(file_path, mode='a') as file:
    file.write(data)

In [29]:
from openai import OpenAI

client = OpenAI(base_url="http://localhost:8080/v1", api_key="lm-studio")

prompt = """
  You are a helpful research financial analyst who speaks and reads spanish.
  You are given a line of textin spanish which is the context. You must generate a question and answer pair.
  You do this for research purposes and to help better understand the text through questions and answers.
  You must generate questions based only on the text you find and you must not use any external information.
  Follow the format of the examples below to include context, question, and answer in the response.

  Example:

  Context: "* Con el objetivo de continuaur con su estrategia de crecimiento ordenada manteniendo un estructura balanceada de capital, Fibra My Ilevo a cabo una Oferta Subsecuente de Capital por Ps. 7,810.4 millones (6,791.7 de oferta base y Ps. 1,018.7 millones de sobre asignacion) los cuales representan aproximadamente USS 467.9 millones de dolares (considerando el tipo de cambio FIX de Ps. 16.6920 al 14 de marzo de 2024) a un precio de 11.50 por OSFL. La emision tuvo una participacion de inversionistas internacionales de 56%, contando con una sobredemanda de 2.1 veces considerando el monto base y 1.9 veeces considerando la sobreasingacion."
  Question: Con qué objetivo llevo una oferta subsecuente de capital?
  Answer: Con el objetivo de continuar su estrategia de crecimiento ordenada.
  Certainty: High

  Context: "* Con el objetivo de continuaur con su estrategia de crecimiento ordenada manteniendo un estructura balanceada de capital, Fibra My Ilevo a cabo una Oferta Subsecuente de Capital por Ps. 7,810.4 millones (6,791.7 de oferta base y Ps. 1,018.7 millones de sobre asignacion) los cuales representan aproximadamente USS 467.9 millones de dolares (considerando el tipo de cambio FIX de Ps. 16.6920 al 14 de marzo de 2024) a un precio de 11.50 por OSFL. La emision tuvo una participacion de inversionistas internacionales de 56%, contando con una sobredemanda de 2.1 veces considerando el monto base y 1.9 veeces considerando la sobreasingacion."
  Question: Cuál fue el monto total de la oferta subsecuente de capital?
  Answer: 7,810.4 millones de pesos.
  Certainty: High

  Generate a question and answer for each of the following lines of text.
  You should return the context you used, which is the text you used to formulate the question, the question, the answer and the certainty level you have about the answer you generated.
  Context, Question, Answer, Certainty must be separated like the following:
  
  "Context: Con el objetivo de continuar con su estrategia de crecimiento ordenada manteniendo un estructura balanceada de capital, Fibra My Ilevo a cabo una Oferta Subsecuente de Capital por Ps. 7,810.4 millones",
  "Question: Con qué objetivo llevo una oferta subsecuente de capital?",
  "Answer: Con el objetivo de continuar su estrategia de crecimiento ordenada.",
  "Certainty: High"
  """

messages = [
  {"role": "system", "content": prompt},
]

file_name = "qa.csv"
write_to_file(file_name, "")

for line in content_lines:
  messages.append({"role": "user", "content": line})

  completion = client.chat.completions.create(
    model="TheBloke/Mistral-7B-Instruct-v0.1-GGUF",
    messages=messages,
    temperature=0.7,
  )

  append_to_file(file_name, completion.choices[0].message.content)
  append_to_file(file_name, "\n--------\n")

KeyboardInterrupt: 